In [1]:
#env: equake_env (Python 3.9)
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [3]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [4]:
#create and connect to a sqlite3 db
path = "../Database/db/earthquake_db"
conn = sqlite3.connect(path)

Adding 1 month of data

In [5]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-06-26T16:59:17.160Z,34.486000,-118.976667,9.80,2.06,ml,37.0,28.00,0.04994,0.340,...,2022-06-26T17:02:58.379Z,"11km NNW of Fillmore, CA",earthquake,0.51,1.19,0.127,28.0,automatic,ci,ci
1,2022-06-26T16:47:58.020Z,33.273000,-116.261500,1.50,1.01,ml,29.0,92.00,0.09781,0.250,...,2022-06-26T16:51:34.854Z,"11km E of Borrego Springs, CA",earthquake,0.47,0.79,0.196,22.0,automatic,ci,ci
2,2022-06-26T16:46:22.712Z,64.323700,-148.091100,14.50,0.90,ml,NaN,NaN,NaN,0.400,...,2022-06-26T16:48:52.937Z,Central Alaska,earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-06-26T16:37:49.274Z,64.016100,-146.450400,10.10,1.00,ml,NaN,NaN,NaN,0.250,...,2022-06-26T16:39:43.942Z,"30 km WSW of Whitestone, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
4,2022-06-26T16:36:35.810Z,33.841167,-118.531167,10.85,2.06,ml,48.0,76.00,0.14380,0.300,...,2022-06-26T16:47:52.390Z,"12km WSW of Manhattan Beach, CA",earthquake,0.46,0.89,0.188,27.0,automatic,ci,ci
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8920,2022-05-27T17:48:10.470Z,36.367333,-120.582667,13.88,1.74,md,37.0,65.00,0.17570,0.100,...,2022-05-28T22:57:17.983Z,"10km SE of New Idria, CA",earthquake,0.22,0.52,0.301,27.0,reviewed,nc,nc
8921,2022-05-27T17:47:29.000Z,37.499667,-118.848500,4.90,0.41,md,9.0,236.00,0.09151,0.050,...,2022-05-27T18:51:14.280Z,"16km WSW of Toms Place, CA",earthquake,1.04,3.06,0.082,8.0,reviewed,nc,nc
8922,2022-05-27T17:41:27.095Z,37.012100,-116.005600,9.00,-0.10,ml,14.0,151.87,0.06900,0.149,...,2022-06-01T20:19:37.745Z,"57 km NNW of Indian Springs, Nevada",earthquake,NaN,1.30,0.270,9.0,reviewed,nn,nn
8923,2022-05-27T17:26:46.473Z,64.933400,-148.478500,19.00,0.60,ml,NaN,NaN,NaN,0.340,...,2022-05-27T17:31:11.409Z,"23 km WNW of Ester, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak


In [6]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

8925

Load from database to update with new rows and remove duplicate rows

In [7]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67031,2022-05-27T17:48:10.470Z,36.367333,-120.582667,13.88,1.74,md,37.0,65.00,0.17570,0.1000,...,2022-05-28T22:57:17.983Z,"10km SE of New Idria, CA",earthquake,0.22,0.52,0.301,27.0,reviewed,nc,nc
67032,2022-05-27T17:47:29.000Z,37.499667,-118.848500,4.90,0.41,md,9.0,236.00,0.09151,0.0500,...,2022-05-27T18:51:14.280Z,"16km WSW of Toms Place, CA",earthquake,1.04,3.06,0.082,8.0,reviewed,nc,nc
67033,2022-05-27T17:41:27.095Z,37.012100,-116.005600,9.00,-0.10,ml,14.0,151.87,0.06900,0.1490,...,2022-06-01T20:19:37.745Z,"57 km NNW of Indian Springs, Nevada",earthquake,NaN,1.30,0.270,9.0,reviewed,nn,nn
67034,2022-05-27T17:26:46.473Z,64.933400,-148.478500,19.00,0.60,ml,NaN,NaN,NaN,0.3400,...,2022-05-27T17:31:11.409Z,"23 km WNW of Ester, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak


In [8]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67028,2022-05-27T17:48:10.470Z,36.367333,-120.582667,13.88,1.74,md,37.0,65.00,0.17570,0.1000,...,2022-05-28T22:57:17.983Z,"10km SE of New Idria, CA",earthquake,0.22,0.52,0.301,27.0,reviewed,nc,nc
67029,2022-05-27T17:47:29.000Z,37.499667,-118.848500,4.90,0.41,md,9.0,236.00,0.09151,0.0500,...,2022-05-27T18:51:14.280Z,"16km WSW of Toms Place, CA",earthquake,1.04,3.06,0.082,8.0,reviewed,nc,nc
67030,2022-05-27T17:41:27.095Z,37.012100,-116.005600,9.00,-0.10,ml,14.0,151.87,0.06900,0.1490,...,2022-06-01T20:19:37.745Z,"57 km NNW of Indian Springs, Nevada",earthquake,NaN,1.30,0.270,9.0,reviewed,nn,nn
67031,2022-05-27T17:26:46.473Z,64.933400,-148.478500,19.00,0.60,ml,NaN,NaN,NaN,0.3400,...,2022-05-27T17:31:11.409Z,"23 km WNW of Ester, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak


In [9]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [10]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [11]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
67006  2022-05-27T19:51:53.123Z -37.743100  179.932800   10.00  4.20      mb   
67015  2022-05-27T19:05:15.699Z  -6.657500  126.943800  406.83  4.00      mb   
67019  2022-05-27T18:47:02.370Z   8.421600 -103.215500   10.00  4.30      mb   
67022  2022-05-27T18:27:13.256Z -10.512700  166.016300  133.47  4.50      mb   
67024  2022-05-27T18:07:26.440Z  38.830667 -122.813167    1.90  0.44      md   

        nst     gap       dmin     rms 

In [12]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
67003  2022-05-27T19:51:53.123Z -37.743100  179.932800   10.00  4.20      mb   
67012  2022-05-27T19:05:15.699Z  -6.657500  126.943800  406.83  4.00      mb   
67016  2022-05-27T18:47:02.370Z   8.421600 -103.215500   10.00  4.30      mb   
67019  2022-05-27T18:27:13.256Z -10.512700  166.016300  133.47  4.50      mb   
67021  2022-05-27T18:07:26.440Z  38.830667 -122.813167    1.90  0.44      md   

        nst     gap       dmin     rms 

In [13]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

157

In [14]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

64154

In [15]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [16]:
# Close connection to SQLite database
conn.close()
connection.close()